# **Hardware: check that the GPU is selected**



In [ ]:
!nvidia-smi

# **Connecting to the Wandb platform** ##

In [ ]:
!pip install wandb
!wandb login

# **Importing libraries** ##



In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler
import matplotlib.pyplot as plt
import cv2
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms.v2 as transforms
from tqdm import tqdm
import wandb
!pip install pytorch_lightning
from torchmetrics.classification import MulticlassConfusionMatrix, Accuracy
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger, TensorBoardLogger , CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from PIL import Image

# **Installation of the ‘benchamark’ library to evaluate results in detail** ##

In [ ]:
!pip install pytorch_bench
from pytorch_bench import benchmark

# **Downloading the training data set** ##


In [ ]:
!rm -rf FIRE
!rm -rf sample_data
!mkdir -p FIRE/train

!wget https://nextcloud.ig.umons.ac.be/s/KaqzczZsXfsnMER/download/FIRE_DATABASE_3.zip
!unzip FIRE_DATABASE_3.zip -d FIRE/train/DB3
!rm FIRE_DATABASE_3.zip

# DB1 : https://nextcloud.ig.umons.ac.be/s/REWbK6K4XRtoeNw/download/FIRE_DATABASE_1.zip
# DB2 : https://nextcloud.ig.umons.ac.be/s/faKyDy7LCxfz9Xk/download/FIRE_DATABASE_2.zip
# DB3 : https://nextcloud.ig.umons.ac.be/s/KaqzczZsXfsnMER/download/FIRE_DATABASE_3.zip

# **Downloading the test data set** ##

In [ ]:
!wget https://nextcloud.ig.umons.ac.be/s/XqEMQtqQNPoG2cY/download/test.zip
!unzip test.zip -d FIRE/test
!rm test.zip

#**Setting up Hyper-parameters(training parameters)**

In [85]:
Train_data_path = "DB3/FIRE_DATABASE_3" # @param ["small","DB1/FIRE_DATABASE_1","DB2/FIRE_DATABASE_2","DB3/FIRE_DATABASE_3"]
Train_data_path = os.path.join('FIRE/train', Train_data_path)
Test_data_path = "test" #@param ["test","test_defi1"]
Test_data_path = os.path.join('FIRE/test', Test_data_path)
Batch_size=128 #@param [8,16,32,64,128,256] {type:"raw"}
Epochs=50 #@param [2,5,10,20,50,100,200] {type:"raw"}
Learning_rate = 0.001 #@param [0.1, 0.01,0.02,0.05,0.001,0.002,0.005] {type:"raw"}
Train_split = 0.7 #@param [0.7,0.8,0.9] {type:"raw"}
Img_size = 224 #@param [224,299] {type:"raw"}
Accelerator= "auto" #@param ["cpu","gpu","auto"]
num_classes = 3
LOG_DIR="logs/"

#**Download VGG16 pre-trained model & view architecture**

In [ ]:
vgg16 = models.vgg16(pretrained=True)

print(vgg16)

# **Define model, Forward/Backward & Transfer Leraning parameters** ##

In [88]:
class FireDetectionModel(pl.LightningModule):
    def __init__(self, num_classes=num_classes, learning_rate=Learning_rate):
        super().__init__()

        self.confusion_matrix = MulticlassConfusionMatrix(num_classes=num_classes)

        self.model_vgg16 = vgg16

        for param in self.model_vgg16.parameters():
            param.requires_grad = False

        self.model_vgg16.classifier[6] = nn.Linear(4096, num_classes)

        self.criterion = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate
        self.num_classes = num_classes
        self.test_accuracy = Accuracy(task="multiclass", num_classes=self.num_classes)
        self.val_accuracy = Accuracy(task="multiclass", num_classes=self.num_classes)
        self.train_accuracy = Accuracy(task="multiclass", num_classes=self.num_classes)


    def forward(self, x):
        return self.model_vgg16(x)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        acc = self.train_accuracy(outputs, labels)

        self.log_dict({'train_loss':loss,"train_acc":acc}, on_step=True,prog_bar=True,logger=True, on_epoch=True)
        return loss

    def on_train_epoch_end(self):
        self.train_accuracy.reset()

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        val_loss = self.criterion(outputs, labels)
        _, predicted = torch.max(outputs, 1)
        val_acc = self.val_accuracy(predicted, labels)
        self.log_dict({'val_loss':val_loss,"val_acc":val_acc},prog_bar=True, on_step=False, on_epoch=True)
        return {'val_loss': val_loss, 'val_acc': val_acc}

    def on_validation_epoch_end(self):
        self.val_accuracy.reset()

    def test_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)
        test_loss = F.cross_entropy(outputs, y)
        test_acc = self.test_accuracy(outputs, y)
        self.log_dict({'test_loss': test_loss, "test_acc": test_acc}, prog_bar = True, on_step = False, on_epoch = True)
        self.confusion_matrix.update(outputs.argmax(dim = 1), y)
        return {'test_loss': test_loss, 'test_acc': test_acc}

    def on_test_end(self):
        self.test_accuracy.reset()
        fig_, ax_ = self.confusion_matrix.plot()
        self.confusion_matrix.reset()

    def configure_optimizers(self):
        #return torch.optim.SGD(self.model.classifier.parameters(), lr=self.learning_rate)
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate, weight_decay=1e-4)


# **Display some images using the ‘display_class_images’ function** ##

In [89]:
def display_class_images(class_path):
  import glob
  import matplotlib.image as mpimg
  images = []
  for img_path in glob.glob(class_path):
      images.append(mpimg.imread(img_path))
  plt.figure(figsize=(14,12))
  columns = 4
  for i, img in enumerate(images):
      if (i<=4):
        img=cv2.resize(img, (256,256))
        plt.subplot(5, 5, i + 1)
        plt.imshow(img)
        plt.axis('off')
  plt.show()

In [ ]:
display_class_images(Train_data_path + "/start_fire/*.jpg")

# **Creation of training data sets, validation and ‘Data Loaders’ testing** #

In [90]:
def create_data_loaders(dataset_path, testset_path, batch_size, train_split, img_size):
    transform = transforms.Compose([
        transforms.RandomResizedCrop(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])

    print(f"Train data path: {Train_data_path}")
    print(f"Test data path: {Test_data_path}")

    dataset = datasets.ImageFolder(dataset_path, transform=transform)
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(train_split * dataset_size))
    np.random.shuffle(indices)
    train_indices, val_indices = indices[:split], indices[split:]

    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)

    test_transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor()
    ])
    test_dataset = datasets.ImageFolder(testset_path, transform=test_transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    print(f"Train size: {len(train_indices)}, Val size: {len(val_indices)}")
    print(f"Dataset path: {dataset_path}, Testset path: {testset_path}")

    return train_loader, val_loader, test_loader

# **Define Hyper-parameters, EarlyStopping, Checkpoints and WandDB parameters** #

In [ ]:
# Initialize wandb
wandb.init(project="fire-detection", config={
    "learning_rate": Learning_rate,
    "epochs": Epochs,
    "batch_size": Batch_size,
    "model": "vgg16"
})

# Create data loaders
train_loader, val_loader, test_loader = create_data_loaders(
    Train_data_path, Test_data_path, Batch_size, Train_split, Img_size
)

# Initialize model
model_vgg16 = FireDetectionModel(num_classes=3, learning_rate=Learning_rate)

# Setup callbacks
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints',
    filename='best-checkpoint',
    save_top_k=1,
    mode='min'
)
early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=10,
    mode='min'
)


# Initialize WandbLogger
wandb_logger = WandbLogger(project="fire-detection")

# Initialize WandbLogger
csv_logger = CSVLogger(LOG_DIR, name="cnn", version='')

# Initialize Trainer
trainer = pl.Trainer(
    max_epochs=Epochs,
    accelerator=Accelerator,
    log_every_n_steps=1,
    devices=1,
    logger=[wandb_logger, csv_logger],
    callbacks=[checkpoint_callback, early_stop_callback],
)

# **Start training** #

In [ ]:
# Train the model
trainer.fit(model_vgg16, train_loader, val_loader)

# **Evaluating the model** ##

In [ ]:
# Test the model
trainer.test(model_vgg16, test_loader)

# **Display training curves using the ‘plot_metrics’ function** ##

In [95]:
def plot_metrics(log_folder):
  import pandas as pd
  import matplotlib.pyplot as plt

  # Load the CSV file generated by CSVLogger
  df = pd.read_csv(f'{LOG_DIR}/{log_folder}/metrics.csv')
  train_df = df[df['train_loss_epoch'].notna()]
  val_df = df[df['val_loss'].notna()]

  # Plot training loss
  plt.plot(train_df['epoch'], train_df['train_loss_epoch'], label='Train Loss')
  plt.plot(val_df['epoch'], val_df['val_loss'], label='Validation Loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.title('Training & Validation Loss')
  plt.legend()
  plt.grid(True)
  plt.show()

  # Plot training accuracy
  plt.plot(train_df['epoch'], train_df['train_acc_epoch'], label='Train Acc')
  plt.plot(val_df['epoch'], val_df['val_acc'], label='Val Acc')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.title('Training & Validation Accuracy')
  plt.legend()
  plt.grid(True)
  plt.show()

In [ ]:
plot_metrics('cnn')

# **Evaluation of the model using various metrics from ‘Benchmark’ library** #

In [ ]:
example_input = torch.randn(1, 3, Img_size, Img_size)
results = benchmark(model_vgg16, example_input)

# log to wandb
wandb.log({
    "benchmark": results
})
wandb.finish()


# **Test the classification model with a test image of your choice"**

In [ ]:
image_path = "YOUR IMAGE"

classes = ["fire", "no fire", "start fire"]

# Load and preprocess the image
img = Image.open(image_path).convert('RGB')
transform = transforms.Compose([
    transforms.Resize((Img_size, Img_size)),
    transforms.ToTensor(),
])
x = transform(img).unsqueeze(0).to("cuda")  # Add batch dimension

model_vgg16.to("cuda")

# Predict
model_vgg16.eval()
with torch.no_grad():
    pred = model_vgg16(x)
    probabilities = torch.nn.functional.softmax(pred[0], dim=0)

# Display results
plt.figure(figsize=(10, 10))
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

for pos, prob in enumerate(probabilities):
    class_name = classes[pos]
    print(f"Class Name: {class_name} --- Class Probability: {prob.item()*100:.2f}%")

    if pos == torch.argmax(probabilities).item():
        font = cv2.FONT_HERSHEY_COMPLEX
        textsize = cv2.getTextSize(class_name, font, 2, 3)[0]
        textX = (img.shape[1] - textsize[0]) // 2
        textY = (img.shape[0] + textsize[1]) // 2
        cv2.putText(img, f"{class_name}: {prob.item()*100:.2f}%", (textX-50, textY), font, 2, (255,0,0), 6, cv2.LINE_AA)

plt.imshow(img)
plt.axis('off')
plt.show()

# **Test the classification model with a test video of your choice"**

In [ ]:
import cv2
import torch
import torchvision.transforms as transforms
from google.colab.patches import cv2_imshow

video_path = "YOUR FILE.mp4"  # Path to the incoming video
output_path = "output_video.mp4"  # Path to the output video
classes = ["fire", "no fire", "start fire"]

# Loading the model
model_vgg16.to("cuda")  # Moving the model to the GPU
model_vgg16.eval()  # Setting to evaluation mode

# Transformations for every frame
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((Img_size, Img_size)),
    transforms.ToTensor(),
])

# Open video file
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Video could not be opened")
    exit()

# Get information about the video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Preparing to record the processed video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Video has ended

    # Frame conversion
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    x = transform(img).unsqueeze(0).to("cuda")  # Adding a batch size

    # Prediction
    with torch.no_grad():
        pred = model_vgg16(x)
        probabilities = torch.nn.functional.softmax(pred[0], dim=0)

    # Determining the class with the highest probability
    predicted_class = torch.argmax(probabilities).item()
    class_name = classes[predicted_class]
    confidence = probabilities[predicted_class].item() * 100

    # Add text to a frame
    font = cv2.FONT_HERSHEY_COMPLEX
    text = f"{class_name}: {confidence:.2f}%"
    cv2.putText(frame, text, (50, 50), font, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Recording the processed frame
    out.write(frame)

# Freeing up resources
cap.release()
out.release()

print("Processing completed.")


# **XAI**

In [ ]:
!pip install captum
!pip install pytorch_lightning

In [49]:
import torch
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms.v2 as transforms
import torch.nn as nn
from captum.attr import LayerGradCam
import matplotlib.pyplot as plt
from torchvision import models
import pytorch_lightning as pl
import numpy as np
import cv2

In [50]:
img_size = 224
testset_path  = "FIRE/test/test/"
batch_size = 1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

test_transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor()
    ])

test_dataset = datasets.ImageFolder(testset_path, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
def get_last_conv_layer(model):
    last_conv_layer = None
    for module in model.modules():
        if isinstance(module, nn.Conv2d):
            last_conv_layer = module
    if last_conv_layer is None:
        raise ValueError("No Conv2d layer found in the model.")
    return last_conv_layer

def visualize_gradcam(model, inputs, labels, device, figsize=(18, 6)):
    # Get last convolutional layer
    last_conv_layer = get_last_conv_layer(model)
    gradcam = LayerGradCam(model, last_conv_layer)

    # Convert labels if necessary
    if isinstance(labels, torch.Tensor):
        labels = int(torch.argmax(labels.detach()).cpu().numpy())

    model_vgg16.to("cuda")

    # Compute Grad-CAM attribution
    attribution = gradcam.attribute(inputs, target=labels, relu_attributions=True)

    # Convert tensors to numpy arrays
    attribution_np = attribution[0].cpu().detach().numpy()
    inputs_np = inputs[0].cpu().permute(1, 2, 0).detach().numpy()

    # Get original image dimensions
    height, width = inputs_np.shape[:2]

    # Resize attribution map to match input image size
    heatmap = cv2.resize(attribution_np[0], (width, height))

    # Normalize heatmap
    heatmap = np.maximum(heatmap, 0)
    heatmap = heatmap / np.max(heatmap)

    # Plot results
    fig, axes = plt.subplots(1, 3, figsize=figsize)

    # Original image
    axes[0].imshow(inputs_np)
    axes[0].set_title('Original Image')
    axes[0].axis('off')

    # Heatmap only
    im = axes[1].imshow(heatmap, cmap='jet')
    axes[1].set_title('Attention Heatmap')
    axes[1].axis('off')
    plt.colorbar(im, ax=axes[1])

    # Overlay
    axes[2].imshow(inputs_np)
    axes[2].imshow(heatmap, alpha=0.5, cmap='jet')
    axes[2].set_title('Overlay')
    axes[2].axis('off')

    plt.tight_layout()
    return fig

#inputs, labels = next(iter(test_loader))
#inputs = inputs.to(device)

index = 271  # Index of the image
inputs, labels = test_dataset[index]
inputs = inputs.unsqueeze(0).to(device)  # Add batch dimension

fig = visualize_gradcam(model_vgg16, inputs, labels, device)
plt.show()

# **Localisation**

In [ ]:
!pip install ultralytics

In [24]:
nb_classes       = 2
batch_size       = 8 #@param [8,16,32,64] {type:"raw"}
epochs           = 5 #@param [5, 10,20,50,100,200] {type:"raw"}
dataset_path     = "/content/fire_detection_dataset/D-Fire"
input_dim        = 640 #@param [640] {type:"raw"}
train_dataset    = "images/train"
test_dataset     = "images/test"
valid_dataset    = "images/valid"
yaml_config_name = 'dataset.yaml'
project_path     = "/content/drive/MyDrive/fire/fire_detection"
yolo_version     = "yolov8s" #@param ["yolov8n", "yolov8s","yolov8m","yolov8l","yolov8x"] {type:"string"}
classes          =  ['smoke', 'fire']


In [ ]:
!wget https://nextcloud.ig.umons.ac.be/s/8QNxNrPEEQyE9tN/download/d_fire.zip
!unzip d_fire.zip -d /content/fire_detection_dataset/

In [26]:
import yaml

def generate_yolo_config(output_path):
    # Define the configuration
    config = {
        "path": dataset_path,  # dataset root dir
        "train": train_dataset,  # train images (relative to 'path')
        "val": valid_dataset,  # val images (relative to 'path')
        "test": test_dataset,
        'names': classes
    }

    # Write the configuration to a .yaml file
    with open(output_path, 'w') as file:
        yaml.dump(config, file, default_flow_style=False)

generate_yolo_config(yaml_config_name)

In [ ]:
from ultralytics import YOLO

model = YOLO(f'{yolo_version}.pt')

# Train model
results = model.train(data='dataset.yaml', epochs=epochs, imgsz=input_dim)

# Evaluate model
results = model.val()

In [ ]:
results = model(f'{dataset_path}/images/train/AoF00015.jpg')

for result in results:
    boxes = result.boxes
    masks = result.masks
    result.show()

# **Test the localisation model with a test video of your choice"**

In [ ]:
import cv2
import torch
from ultralytics import YOLO

video_path = "YOUR FILE"  # Path to the incoming video
output_path = "output_yolo.mp4"  # Path to the output video
classes = ["smoke", "fire"]  # Classes

# Завантаження моделі
model.to("cuda")  # Moving the model to the GPU

# Open a video file
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Video could not be opened")
    exit()

# Get information about a video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Preparing to record the processed video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Video has ended

    # YoloV8 prediction
    results = model(frame, device="cuda")  # Transferring the frame to the inferno

    # Getting predictions
    for result in results:
        boxes = result.boxes  # Bounding rectangles
        for box in boxes:
            # Getting coordinates, class, and probability
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Rectangle coordinates
            cls = int(box.cls[0])  # Class
            confidence = float(box.conf[0]) * 100  # Probability

            # Add text and rectangle to a frame
            label = f"{classes[cls]}: {confidence:.2f}%"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Rectangle
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Recording the processed frame
    out.write(frame)

# Freeing up resources
cap.release()
out.release()

print("Processing completed.")


# **Test the classification + localisation model with a test video of your choice"**

In [ ]:
import cv2
import torch
from ultralytics import YOLO

video_path = "start_fire.mp4"  # Path to the incoming video
output_path = "out_fire_smoke(1).mp4"  # Path to the output video

classes_vgg16 = ["fire", "no fire", "start fire"]  # VGG16 classes
classes_yolo = ["smoke", "fire"]  # YoloV8 classes

# Loading model
model_vgg16.to("cuda")  # Moving the model to the GPU
model.to("cuda")  # Moving the model to the GPU

model_vgg16.eval()

# Transformations for every frame
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((Img_size, Img_size)),
    transforms.ToTensor(),
])

# Open a video file
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Video could not be opened")
    exit()

# Get information about a video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Preparing to record the processed video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Video has ended

    # Frame conversion
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    x = transform(img).unsqueeze(0).to("cuda")  # Adding a batch size

    # Prediction
    with torch.no_grad():
        pred = model_vgg16(x)
        probabilities = torch.nn.functional.softmax(pred[0], dim=0)

    # Determining the class with the highest probability
    predicted_class = torch.argmax(probabilities).item()
    class_name = classes_vgg16[predicted_class]
    confidence = probabilities[predicted_class].item() * 100

    # Add text to a frame
    font = cv2.FONT_HERSHEY_COMPLEX
    text = f"{class_name}: {confidence:.2f}%"
    cv2.putText(frame, text, (50, 50), font, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # YoloV8 predictions
    results = model(frame, device="cuda")  # Transferring the frame to the inferno

    # Getting predictions
    for result in results:
        boxes = result.boxes  # Bounding rectangles
        for box in boxes:
            # Getting coordinates, class, and probability
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Rectangle coordinates
            cls = int(box.cls[0])  # Клас
            confidence = float(box.conf[0]) * 100  # Probability

            # Add a rectangle and text
            label = f"{classes_yolo[cls]}: {confidence:.2f}%"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Rectangle
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Recording the processed frame
    out.write(frame)

# Freeing up resources
cap.release()
out.release()

print("Processing completed.")
